In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook

In [2]:
import sys
sys.path.append("../../odeon/")


In [3]:
import odeon
from odeon.models.opencd import OpenCDPlug

/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
ocd = OpenCDPlug(
    config_file="../../open-cd/configs/changer/changer_ex_s50_512x512_40k_levircd.py",
    loss="ce"
)

10/26 10:39:47 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
    CUDA available: False
    numpy_random_seed: 384723702
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.0
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.1-Product Build 20220311 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.7, CUDNN_VERSION=8.5.0, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -Wno-deprec

10/26 10:39:48 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/26 10:39:48 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/mmseg/models/losses/cross_entropy_loss.py:249: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
/home/PVoitot/mamb

In [11]:
levir_root = "/mnt/store_dai/datasrc/dchan/levir-cd-repatched-512x512"

import glob
from pathlib import Path
import csv
from matplotlib import pyplot as plt
#plt.ioff()

import odeon
from odeon.data.data_module import Input

levir_path = Path(levir_root)

batch_size = 10
num_workers = 8

train_params = {
    'input_fields': {
        "T0": {"name": "A", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "T1": {"name": "B", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "mask": {"name": "label", "type": "mask", "post_process": "floor_div", "dtype": "uint8"},
    },
    'dataloader_options' : {"batch_size": batch_size, "num_workers": num_workers},
    'input_file': levir_path / "train.csv",
    'input_files_has_header': 'infer',
    'root_dir': levir_path,
    'nb_samples': 0,
    'sample_seed': 0
}

val_params = {
    'input_fields': {
        "T0": {"name": "A", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "T1": {"name": "B", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "mask": {"name": "label", "type": "mask", "post_process": "floor_div", "dtype": "uint8"},       
    },
    'dataloader_options' : {"batch_size": batch_size, "num_workers": num_workers},
    'input_file': levir_path / "val.csv",
    'input_files_has_header': 'infer',
    'root_dir': levir_path,
    'nb_samples': 0,
    'sample_seed': 0
}

test_params = {
    'input_fields': {
        "T0": {"name": "A", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "T1": {"name": "B", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3], "mean": [0.485, 0.456, 0.406], "std": [0.228, 0.224, 0.225]},
        "mask": {"name": "label", "type": "mask", "post_process": "floor_div", "dtype": "uint8"},
    },
    'dataloader_options' : {"batch_size": batch_size, "num_workers": num_workers},
    'input_file': levir_path / "test.csv",
    'root_dir': levir_path,
    'input_files_has_header': 'infer',
    'nb_samples': 0,
    'sample_seed': 0
}


input = Input(
    fit_params=train_params,
    validate_params=val_params,
    test_params=test_params
)


In [12]:
input.prepare_data()
input.setup(stage="fit")

In [13]:
v = input.fit.dataset[0]
v["mask"].unique()

/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


tensor([0., 1.], dtype=torch.float64)

In [ ]:
for idx, b in enumerate(input.fit.dataloader):
    print(b["mask"].unique())
    o = ocd.training_step(b, idx)
    print(o)
    if idx > 1:
        break

tensor([0., 1.], dtype=torch.float64)


/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/pytorch_lightning/core/module.py:416: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


{'loss': tensor(0.7472, grad_fn=<NllLoss2DBackward0>)}
tensor([0., 1.], dtype=torch.float64)
